# Tutorial on customizing unit models in WatertAP
Demonstration how to modify existing unit models at flowsheet level

## Dependencies
* Python - Programming language
* Pyomo - Python package for equation-oriented modeling
* IDAES - Python package extending Pyomo for flowsheet modeling
* WaterTAP - Unit models

## Demonstration structure 
* Setting up basic RO flowsheet
* Replace a fixed variable with an equation
* Replace existing constraint with a new one for compartive analysis  

## Flowsheet considered in the example
<img src="RO_flowsheet.png" width="500" height="200">

In [334]:
## Import core components 
# Pyomo cor ecomponents
from pyomo.environ import (Param,Var, Constraint, TransformationFactory, Reals,    ConcreteModel,
    value,assert_optimal_termination,
    units as pyunits)
from pyomo.network import Arc
# Ideas core comoponents
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.solvers import get_solver
from idaes.core.util.scaling import constraint_scaling_transform
from idaes.core.util.initialization import propagate_state
from idaes.models.unit_models import Feed
# WaterTAP core components 
import watertap.property_models.seawater_prop_pack as properties
from watertap.unit_models.reverse_osmosis_1D import (
    ReverseOsmosis1D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType,
)
from watertap.unit_models.pressure_changer import Pump


### Define a default build function for unit model

In [335]:
def build_ro():
    m = ConcreteModel()
    # create IDAES flowsheet
    m.fs = FlowsheetBlock(dynamic=False)
    # create seawater property model
    m.fs.properties = properties.SeawaterParameterBlock()
    
    # build feed
    m.fs.feed = Feed(property_package=m.fs.properties)
    #build pump
    m.fs.pump = Pump(property_package=m.fs.properties)
    m.fs.RO = ReverseOsmosis1D(
        property_package=m.fs.properties,
        has_pressure_change=True,
        pressure_change_type=PressureChangeType.calculated,
        mass_transfer_coefficient=MassTransferCoefficient.calculated,
        concentration_polarization_type=ConcentrationPolarizationType.calculated,
        transformation_scheme="BACKWARD",
        transformation_method="dae.finite_difference",
        finite_elements=10,
    )
    # connect feed to pump
    m.fs.feed_to_pump = Arc(source=m.fs.feed.outlet, destination = m.fs.pump.inlet)
    #connect pump to RO unit
    m.fs.pump_to_ro = Arc(source=m.fs.pump.outlet, destination = m.fs.RO.inlet)
    TransformationFactory("network.expand_arcs").apply_to(m)
    return m


### Define function to set default values for flowsheet and calculate scaling factors    

In [336]:
def set_default_operation(m):
    m.fs.feed.properties[0].temperature.fix(273 + 25)                      # temperature (K)
    m.fs.feed.properties[0].pressure.fix(101325)                           # pressure (Pa)
    m.fs.feed.properties[0].flow_mass_phase_comp['Liq', 'H2O'].fix(0.965)  # mass flowrate of H2O (kg/s)
    m.fs.feed.properties[0].flow_mass_phase_comp['Liq', 'TDS'].fix(0.035)  # mass flowrate of TDS (kg/s)
    m.fs.properties.set_default_scaling(
        "flow_mass_phase_comp",
        1/0.965,
        index=("Liq", "H2O"),
    )
    m.fs.properties.set_default_scaling(
        "flow_mass_phase_comp",
        1/0.035,
        index=("Liq", "TDS"),
    )
    # to help with initialization lets build OSM variable on feed block
    # which we can use to guess operating pressure for RO unit and set pump pressure during initialization
    m.fs.feed.properties[0].pressure_osm_phase[...]
    # define pump defaults
    m.fs.pump.efficiency_pump[0].fix(0.75)
    # scale work and pressures for the pump
    set_scaling_factor(m.fs.pump.control_volume.work, 1e-4)
    set_scaling_factor(m.fs.pump.control_volume.properties_out[0].pressure, 1e-5)
    set_scaling_factor(m.fs.pump.control_volume.properties_in[0].pressure, 1e-5)

    # to help with initalization lets build OSM variable on pump outlet
    # which we can use to guess operating pressure for RO unit 

    # define RO default values for initialization 
    # we opt to specify stage area, and inlet velocity
    # unfixing width and area
    # We also apply variable scaling as we set up each default parameters 
    
    m.fs.RO.feed_side.velocity[0, 0].fix(0.1)

    m.fs.RO.area.fix(50)
    set_scaling_factor(m.fs.RO.area,1/50)
    m.fs.RO.length.unfix()
    set_scaling_factor(m.fs.RO.length, 0.1)
    m.fs.RO.width.unfix()
    set_scaling_factor(m.fs.RO.width, 0.1)

    # we need to specify RO permeate pressure
    m.fs.RO.permeate.pressure[0].fix(101325)
    # we need to specify default values for default mass transport correlation
    # and friction factor correlations 
    m.fs.RO.feed_side.channel_height.fix(1 / 10 / 100)
    m.fs.RO.feed_side.spacer_porosity.fix(0.9)

    # Specify default A and B values, these are dfined in m/s at unit level. 
    m.fs.RO.A_comp[0, "H2O"].fix(3 / (3600 * 1000 * 1e5))
    m.fs.RO.B_comp[0, "TDS"].fix(0.15 / (3600 * 1000))

    # calculate all the scailing factors 
    calculate_scaling_factors(m)


### Define a default init routine for feed and pump 

In [337]:
def init_feed_and_pump(m, solver):

    m.fs.feed.initialize(optarg=solver.options)
    propagate_state(m.fs.feed_to_pump)
    # get osmotic pressure
    osmotic_feed_pressure=value(m.fs.feed.properties[0].pressure_osm_phase['Liq'])
    print("Osmotic pressure is {} bar".format(osmotic_feed_pressure/1e5))
    m.fs.pump.outlet.pressure[0].fix(osmotic_feed_pressure*1.5) 
    m.fs.pump.initialize(optarg=solver.options)
    propagate_state(m.fs.pump_to_ro)

### Lets build, and intialize our model 

In [338]:
m=build_ro()
set_default_operation(m)
solver = get_solver() # get solver
init_feed_and_pump(m,solver)
# Now we can initialize our RO unit 
m.fs.RO.initialize(optarg=solver.options)
# Check degrees of freedom 
print('We have {} degrees of freedom and expect 0'.format(degrees_of_freedom(m)))
assert degrees_of_freedom(m) == 0

result =solver.solve(m, tee=True)
assert_optimal_termination(result)


2023-10-02 00:57:28 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.RO.feed_side.area
2023-10-02 00:57:29 [INFO] idaes.init.fs.feed.properties: fs.feed.properties State Released.
2023-10-02 00:57:29 [INFO] idaes.init.fs.feed: Initialization Complete.
Osmotic pressure is 25.869850382106517 bar
2023-10-02 00:57:29 [INFO] idaes.init.fs.pump.control_volume.properties_out: fs.pump.control_volume.properties_out State Released.
2023-10-02 00:57:29 [INFO] idaes.init.fs.pump.control_volume: Initialization Complete
2023-10-02 00:57:29 [INFO] idaes.init.fs.pump.control_volume.properties_in: fs.pump.control_volume.properties_in State Released.
2023-10-02 00:57:29 [INFO] idaes.init.fs.pump: Initialization Complete: optimal - Optimal Solution Found
2023-10-02 00:57:29 [INFO] idaes.init.fs.RO.feed_side: Initialization Complete
2023-10-02 00:57:29 [INFO] idaes.init.fs.RO.feed_side.properties: fs.RO.feed_side.properties State Released.
2023-10-02 00:57:29 [INFO] idaes.init.fs.RO.feed_s

### Create utility function to display RO performance

In [339]:
def display_ro(m):
    m.fs.RO.report()
    print(
            "RO water flux is {} LMH".format(value(m.fs.RO.flux_mass_phase_comp_avg[0, "Liq", "H2O"] * 3.6e3))
        )

    R=value((m.fs.RO.feed_side.properties[0.0,0.0].conc_mass_phase_comp["Liq", "TDS"] - m.fs.RO.mixed_permeate[0].conc_mass_phase_comp["Liq", "TDS"])/m.fs.RO.feed_side.properties[0.0,0.0].conc_mass_phase_comp["Liq", "TDS"])*100
    print(
            "RO rejection is {} %".format(R)
        )
    print('RO A value is {}'.format(m.fs.RO.A_comp[0, "H2O"].value * (3600 * 1000 * 1e5)))
display_ro(m)


Unit : fs.RO                                                               Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                        : Value   : Units         : Fixed : Bounds
                 Membrane Area :  50.000 :    meter ** 2 :  True : (0.1, 100000.0)
    Solvent Mass Recovery Rate : 0.19471 : dimensionless : False : (0.01, 0.999999)
      Volumetric Recovery Rate : 0.19294 : dimensionless : False : (0.01, 0.999999)

------------------------------------------------------------------------------------
    Stream Table
                                              Units         Feed Inlet  Feed Outlet  Permeate Outlet
    flow_mass_phase_comp ('Liq', 'H2O')  kilogram / second     0.96500     0.77711        0.18789   
    flow_mass_phase_comp ('Liq', 'TDS')  kilogram / second    0.035000    0.034905     9.4739e-05   
    temperature                                     kelvin 

### Add relationship between A parameter and inlet pressure to account for compaction effects    

For P<80 bar A=3 LMH/bar
For P>80 A = A_ini*80/P_inlet  

In [340]:
# Imports smooth min and smooth max functions
import idaes.core.util.math as idaesMath

m.fs.A_var_initial=Var(initialize=3.0)
m.fs.A_var_initial.fix()
set_scaling_factor(m.fs.A_var_initial, 1/m.fs.A_var_initial.value)
m.fs.RO.A_pressure_constraint=(
    Constraint(expr=m.fs.RO.A_comp[0, "H2O"]*(3600 * 1000 * 1e5)==
    idaesMath.smooth_min(m.fs.A_var_initial,(m.fs.A_var_initial*80*1e5/m.fs.RO.inlet.pressure[0]))))
m.fs.RO.A_comp[0,'H2O'].unfix()
m.fs.RO.A_pressure_constraint.pprint()

A_pressure_constraint : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                                                                                                                                                                                                                     : Upper : Active
    None :   0.0 : 360000000000.0*fs.RO.A_comp[0.0,H2O] - 0.5*(fs.A_var_initial + 8000000.0*fs.A_var_initial/fs.RO.feed_side.properties[0.0,0.0].pressure - ((fs.A_var_initial - 8000000.0*fs.A_var_initial/fs.RO.feed_side.properties[0.0,0.0].pressure)**2 + 1e-08)**0.5) :   0.0 :   True


### Verify that constraint produces expected outcomes 

In [341]:
# import method to calcualte variable from constraint, allows evaluation of our constraint 
from pyomo.util.calc_var_value import calculate_variable_from_constraint

for pressure in [10,80,85,90,100,200,300]:
    m.fs.RO.inlet.pressure[0]=pressure*1e5 # needs to be in kPa
    calculate_variable_from_constraint(m.fs.RO.A_comp[0, "H2O"],m.fs.RO.A_pressure_constraint)
    print('Pressure is {} and A value is {}'.format(value(m.fs.RO.inlet.pressure[0])/1e5,value(m.fs.RO.A_comp[0, "H2O"]*(3600 * 1000 * 1e5))))

Pressure is 10.0 and A value is 2.9999999998809526
Pressure is 80.0 and A value is 2.99995
Pressure is 85.0 and A value is 2.823529397598041
Pressure is 90.0 and A value is 2.6666666591666663
Pressure is 100.0 and A value is 2.3999999958333333
Pressure is 200.0 and A value is 1.1999999986111112
Pressure is 300.0 and A value is 0.7999999988636362


### Initalize value at operating pressure and solve model with new constraint

In [342]:
# lets intialize the A value to our actual operating pressure 
m.fs.RO.inlet.pressure[0]=m.fs.pump.outlet.pressure[0].value 
calculate_variable_from_constraint(m.fs.RO.A_comp[0, "H2O"],m.fs.RO.A_pressure_constraint)

print('We have {} degrees of freedom and expect 0'.format(degrees_of_freedom(m)))
assert degrees_of_freedom(m) == 0

result =solver.solve(m, tee=True)
assert_optimal_termination(result)

display_ro(m)

We have 0 degrees of freedom and expect 0
ipopt-watertap: Ipopt with user variable scaling and IDAES jacobian constraint scaling
Ipopt 3.13.2: tol=1e-08
constr_viol_tol=1e-08
nlp_scaling_method=user-scaling
bound_relax_factor=0.0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    pu

### Increase pressure to observe decrease in A value 

In [343]:
m.fs.pump.outlet.pressure[0].fix(1e5*125) # 100 bar
result =solver.solve(m, tee=True)
assert_optimal_termination(result)

display_ro(m)

ipopt-watertap: Ipopt with user variable scaling and IDAES jacobian constraint scaling
Ipopt 3.13.2: tol=1e-08
constr_viol_tol=1e-08
nlp_scaling_method=user-scaling
bound_relax_factor=0.0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the

### Replacing an existing constraint with a new one

A common uncertainty in membrane process is prediction of mass transport rate 
RO model uses a corelation derived by Guilen et al from CFD simulation of 2D RO flow channel with a circular spacers
but a large number of other correlations exist 

Current corelation is  Sh = 0.45* (Re*Sc)^0.36 

@self.Constraint(
    self.flowsheet().config.time,
    self.length_domain,
    self.config.property_package.solute_set,
    doc="Sherwood number",
)
def eq_N_Sh_comp(b, t, x, j):
    return (
        b.N_Sh_comp[t, x, j]
        == 0.46 * (b.N_Re[t, x] * b.N_Sc_comp[t, x, j]) ** 0.36
    )

An alternative corelation that could be used has been derived by Schock & Miquel [Desalination, 1987, 64, 339-352] from experiments. This corelation has been shown to be potentially more accurate then the Hoek correlation [Dudchenko et al. ACS ES&T Engineering, https://doi.org/10.1021/acsestengg.1c00496]

Lets implement the Schock & Miquel correlation instead 

Sh = 0.065 * Re ^ 0.875 * Sc^ 0.33

In [344]:
# Deactivate old constraint 
m.fs.RO.feed_side.eq_N_Sh_comp.deactivate()
m.fs.SH_adjustment=Var(initialize=1)
m.fs.SH_adjustment.fix()
@m.fs.RO.feed_side.Constraint(
    [0],
    m.fs.RO.length_domain,
    m.fs.properties.solute_set,
    doc="Sherwood number Schock & Miquel",
)
def eq_N_Sh_comp_S_and_M(b, t, x, j):
    return (
        b.N_Sh_comp[t, x, j]
        == (0.065 * b.N_Re[t, x]**0.875 * b.N_Sc_comp[t, x, j] ** 0.33) *m.fs.SH_adjustment
    )


### Intialize our new constraint, and solve 

In [345]:
for sh in m.fs.RO.feed_side.N_Sh_comp:
    old_sh_value=(m.fs.RO.feed_side.N_Sh_comp[sh].value)
    calculate_variable_from_constraint(m.fs.RO.feed_side.N_Sh_comp[sh],m.fs.RO.feed_side.eq_N_Sh_comp_S_and_M[sh])
    print('Old Sh {} New Sh value {}'.format(old_sh_value, m.fs.RO.feed_side.N_Sh_comp[sh].value))

Old Sh 27.645006378637014 New Sh value 40.52253619632639
Old Sh 26.35114136602522 New Sh value 35.90636047416856
Old Sh 25.115918745650266 New Sh value 31.799519667245917
Old Sh 23.95677785474015 New Sh value 28.207639405314218
Old Sh 22.884378394240873 New Sh value 25.106431713789792
Old Sh 21.906532060393307 New Sh value 22.461080991038234
Old Sh 21.030649408084074 New Sh value 20.236428115604305
Old Sh 20.260212045925442 New Sh value 18.390453547589203
Old Sh 19.592597967180527 New Sh value 16.873004333602363
Old Sh 19.022269731611264 New Sh value 15.635847815706628
Old Sh 18.54193257258013 New Sh value 14.635353720238276


In [346]:
m.fs.pump.outlet.pressure[0].fix(1e5*125) # 100 bar\
print('----------old result------------')
display_ro(m)
result =solver.solve(m, tee=False)
assert_optimal_termination(result)
print('----------new result------------')
display_ro(m)

----------old result------------

Unit : fs.RO                                                               Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                        : Value   : Units         : Fixed : Bounds
                 Membrane Area :  50.000 :    meter ** 2 :  True : (0.1, 100000.0)
    Solvent Mass Recovery Rate : 0.67061 : dimensionless : False : (0.01, 0.999999)
      Volumetric Recovery Rate : 0.66451 : dimensionless : False : (0.01, 0.999999)

------------------------------------------------------------------------------------
    Stream Table
                                              Units         Feed Inlet  Feed Outlet  Permeate Outlet
    flow_mass_phase_comp ('Liq', 'H2O')  kilogram / second     0.96500     0.31786        0.64714   
    flow_mass_phase_comp ('Liq', 'TDS')  kilogram / second    0.035000    0.034745     0.00025476   
    temperature           

### Lets explore hwo changing sherwood number impacts result

In [348]:
m.fs.SH_adjustment.fix(0.5)
print('----------old result------------')
display_ro(m)
result =solver.solve(m, tee=False)
assert_optimal_termination(result)
print('----------new result 0.5 ------------')
display_ro(m)
m.fs.SH_adjustment.fix(0.25)
result =solver.solve(m, tee=False)
assert_optimal_termination(result)
print('----------new result 0.1 ------------')
display_ro(m)

----------old result------------

Unit : fs.RO                                                               Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                        : Value   : Units         : Fixed : Bounds
                 Membrane Area :  50.000 :    meter ** 2 :  True : (0.1, 100000.0)
    Solvent Mass Recovery Rate : 0.44116 : dimensionless : False : (0.01, 0.999999)
      Volumetric Recovery Rate : 0.43718 : dimensionless : False : (0.01, 0.999999)

------------------------------------------------------------------------------------
    Stream Table
                                              Units         Feed Inlet  Feed Outlet  Permeate Outlet
    flow_mass_phase_comp ('Liq', 'H2O')  kilogram / second     0.96500     0.53928        0.42572   
    flow_mass_phase_comp ('Liq', 'TDS')  kilogram / second    0.035000    0.034728     0.00027215   
    temperature           